<a href="https://colab.research.google.com/github/osfieldgaga/ac_controller_pmv_ml/blob/main/Thermal_comfort_prediction_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install micromlgen

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import median_absolute_error
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from micromlgen import port
from sklearn.tree import DecisionTreeRegressor  
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import random

Import the data set you'll work with. We're using the ASHRAE Thermal Comfort Database

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving ashrae_db2.csv to ashrae_db2.csv


In [ ]:
import io
df = pd.read_csv(io.BytesIO(uploaded['ashrae_db2.csv']))

# stuff i don't need 
labels_to_drop = ['Publication (Citation)', 'Data contributor', 'Year', 'Season', 'City', 
                  'Country', 'Building type','Cooling startegy_operation mode for MM buildings',
                  'Heating strategy_building level', 'Age', 'Sex','Air movement acceptability',
                 'Air movement preference', 'SET', 'Met', 'activity_10',
                 'activity_20', 'activity_30', 'activity_60', 'Air temperature (F)',
                 'Ta_h (C)', 'Ta_h (F)', 'Ta_m (C)', 'Ta_m (F)', 'Ta_l (C)', 'Ta_l (F)',
                 'Operative temperature (C)', 'Operative temperature (F)','Radiant temperature (C)',
                 'Radiant temperature (F)', 'Globe temperature (C)', 'Globe temperature (F)',
                 'Tg_h (C)', 'Tg_h (F)', 'Tg_m (C)', 'Tg_m (F)', 'Tg_l (C)', 'Tg_l (F)',
                  'Air velocity (fpm)', 'Velocity_h (m/s)', 'Velocity_h (fpm)', 
                  'Velocity_m (m/s)', 'Velocity_m (fpm)', 'Velocity_l (m/s)', 'Velocity_l (fpm)',
                  'Subject«s height (cm)', 'Subject«s weight (kg)', 'Blind (curtain)',
                  'Fan', 'Window', 'Door', 'Heater',  'Database', 'Climate', 'Humidity preference',
                 'Humidity sensation', 'Outdoor monthly air temperature (F)', 'Thermal sensation',
                 'Thermal sensation acceptability', 'Thermal preference'] 

df_lite = df.drop(labels_to_drop, axis = 1) #drop it
# remove the rows without a PMV
df_lite.dropna(axis = 0, subset = ['PMV'])
df_lite.dropna(axis = 0, subset = ['Air temperature (C)'])

# select only the rows that contain the relevant type of data
# df_lite = df_lite[(df_lite['Koppen climate classification'].str.contains('Aw') == True) |
#                    (df_lite['Koppen climate classification'].str.contains('Af') == True) |
#                     (df_lite['Koppen climate classification'].str.contains('Am') == True) ]
df_lite = df_lite[df_lite['Cooling startegy_building level'].str.contains('Air Conditioned') == True]
df_lite = df_lite[df_lite['PPD'] <= 30]
#df_lite = df_lite[df_lite['Thermal comfort'] > 2]
df_lite = df_lite[(df_lite['PMV'] >= -3) & (df_lite['PMV'] <= 3)]

df_lite

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10,11,13,18,19,34,50) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Koppen climate classification,Cooling startegy_building level,Thermal comfort,PMV,PPD,Clo,Air temperature (C),Relative humidity (%),Air velocity (m/s),Outdoor monthly air temperature (C)
75,Cfa,Air Conditioned,5.2,-0.10,5.00,0.57,25.7,33.1,0.09,32.8
76,Cfa,Air Conditioned,2.5,-0.40,8.00,0.57,24.6,34.9,0.06,32.8
77,Cfa,Air Conditioned,2.7,-0.30,7.00,0.57,26.4,31.7,0.13,32.8
78,Cfa,Air Conditioned,2.2,-0.40,9.00,0.57,25.0,33.3,0.07,32.8
79,Cfa,Air Conditioned,2.3,0.00,5.00,0.57,26.2,30.8,0.08,32.8
...,...,...,...,...,...,...,...,...,...,...
104034,BSh,Air Conditioned,6,-0.03,5.01,0.41,23.6,34.3,0.15,NaN
104035,BSh,Air Conditioned,4,-0.14,5.42,0.45,23.0,35.1,0.17,NaN
104036,BSh,Air Conditioned,5,-0.35,7.59,0.24,23.4,34.2,0.07,NaN
104037,BSh,Air Conditioned,5,-0.33,7.29,0.29,24.2,32.4,0.11,NaN


# Values to predict
We're looking at the temperature and humidity to predict PMV


In [ ]:
y = df_lite['PMV']

features = ['Air temperature (C)', 'Relative humidity (%)', 'Clo', 'Outdoor monthly air temperature (C)']
X = df_lite[features]
X

,Air temperature (C),Relative humidity (%),Clo,Outdoor monthly air temperature (C)
75,25.7,33.1,0.57,32.8
76,24.6,34.9,0.57,32.8
77,26.4,31.7,0.57,32.8
78,25.0,33.3,0.57,32.8
79,26.2,30.8,0.57,32.8
...,...,...,...,...
104034,23.6,34.3,0.41,NaN
104035,23.0,35.1,0.45,NaN
104036,23.4,34.2,0.24,NaN
104037,24.2,32.4,0.29,NaN


# Deal with the missing values
Use the imputer to replace missing values

In [ ]:
my_imputer = SimpleImputer()
imputed_X = pd.DataFrame(my_imputer.fit_transform(X))

# Imputation removed column names, put them back
imputed_X.columns = X.columns

train_X, val_X, train_y, val_y = train_test_split(imputed_X, y, random_state = 0, train_size = 0.85)
imputed_X

,Air temperature (C),Relative humidity (%),Clo,Outdoor monthly air temperature (C)
0,25.7,33.1,0.57,32.800000
1,24.6,34.9,0.57,32.800000
2,26.4,31.7,0.57,32.800000
3,25.0,33.3,0.57,32.800000
4,26.2,30.8,0.57,32.800000
...,...,...,...,...
21331,23.6,34.3,0.41,13.527985
21332,23.0,35.1,0.45,13.527985
21333,23.4,34.2,0.24,13.527985
21334,24.2,32.4,0.29,13.527985


# Use a DecisionTree Regressor

In [ ]:
pmv_tree_model = DecisionTreeRegressor(random_state = 0)
pmv_tree_model.fit (train_X, train_y)
predictions_tree = pmv_tree_model.predict(val_X)

print('Mean error using Decision Tree')
print(mean_absolute_error(val_y, predictions_tree))

print('Mean squared error using Decision Tree')
print(mean_squared_error(val_y, predictions_tree))

print('Median absolute error using Decision Tree')
print(median_absolute_error(val_y, predictions_tree))

Mean error using Decision Tree
0.32298403772630585
Mean squared error using Decision Tree
0.19230675390870944
Median absolute error using Decision Tree
0.25999999999999995


# Use a RandomForest Regressor

In [ ]:
pmv_forest_model = RandomForestRegressor(random_state=0)
pmv_forest_model.fit (train_X, train_y)
predictions2 = pmv_forest_model.predict(val_X)

print('Mean error using Random Forest')
print(mean_absolute_error(val_y, predictions2))

print('Mean squared error using Random Forest')
print(mean_squared_error(val_y, predictions2))

print('Median absolute error using Random Forest')
print(median_absolute_error(val_y, predictions2))

Mean error using Random Forest
0.24687817244027474
Mean squared error using Random Forest
0.10648935580394786
Median absolute error using Random Forest
0.19230000000000014


#  Use RandomizedSearchCV or GridSearchCV to fine tune the parameters

In [ ]:
max_val = 10
param_dist = {"max_depth": [10, 15, 20, 50],
              "min_samples_leaf": [9, 10, 20],
              "criterion": ["squared_error", "friedman_mse", "absolute_error", "poisson"],
              "min_samples_split": [9, 10, 20],
              "max_leaf_nodes": [None, 2, 5, 10, 15, 20, 50]}

tree = DecisionTreeRegressor()
 
# Instantiating RandomizedSearchCV object
# tree_cv = RandomizedSearchCV(tree, param_dist, cv = 5, verbose = True)
tree_cv = GridSearchCV(tree, param_dist, cv = 3, verbose = True, scoring = ["neg_mean_squared_error", "neg_mean_absolute_error"], refit = "neg_mean_absolute_error")
 
tree_cv.fit(train_X, train_y)
 
# Print the tuned parameters and score
print("Tuned Decision Tree Parameters: {}".format(tree_cv.best_params_))
print("Best score is {}".format(tree_cv.best_score_))

Fitting 3 folds for each of 82944 candidates, totalling 248832 fits


# Use the parameters found

From the previous one, the best parameters were
```
Tuned Decision Tree Parameters: {'criterion': 'squared_error', 'max_depth': 10, 'max_leaf_nodes': None, 'min_samples_leaf': 9, 'min_samples_split': 9}
Best score is -0.12044286212891644
```
using parameters


```
max_val = 20
param_dist = {"max_depth": [None, 1, 5, 10, 20],
              "min_samples_leaf": range(2, max_val),
              "criterion": ["squared_error", "friedman_mse", "absolute_error", "poisson"],
              "min_samples_split": range(2, max_val),
              "max_leaf_nodes": [None, 2, 5, 10, 20]}
```





In [ ]:
max_val = 5
param_dist = {"max_depth": [10, 15, 20, 50],
              "min_samples_leaf": [9, 10, 20],
              "criterion": ["squared_error", "friedman_mse", "absolute_error", "poisson"],
              "min_samples_split": [9, 10, 20],
              "max_leaf_nodes": [None, 2, 5, 10, 15, 20, 50]}

tree = DecisionTreeRegressor()
 
# Instantiating RandomizedSearchCV object
# tree_cv = RandomizedSearchCV(tree, param_dist, cv = 5, verbose = True)
tree_cv = GridSearchCV(tree, param_dist, cv = 3, verbose = True, scoring = "neg_mean_absolute_error")
 
tree_cv.fit(train_X, train_y)
 
# Print the tuned parameters and score
print("Tuned Decision Tree Parameters: {}".format(tree_cv.best_params_))
print("Best score is {}".format(tree_cv.best_score_))

Fitting 3 folds for each of 1008 candidates, totalling 3024 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
756 fits failed out of a total of 3024.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
756 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 1320, in fit
    X_idx_sorted=X_idx_sorted,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 179, in fit
    "Some value(s) of y are negative which is"
ValueError: Some value(s) of y are negative which is 

Tuned Decision Tree Parameters: {'criterion': 'absolute_error', 'max_depth': 10, 'max_leaf_nodes': None, 'min_samples_leaf': 9, 'min_samples_split': 20}
Best score is -0.25851419906258616


In [ ]:
pmv_tree_model_optimized = DecisionTreeRegressor(random_state = 0, criterion ='squared_error', 
                                                 max_depth = 10, max_leaf_nodes = None, min_samples_leaf = 9, min_samples_split = 9)
pmv_tree_model_optimized.fit (train_X, train_y)
predictions_tree = pmv_tree_model_optimized.predict(val_X)

print('Mean error using Decision Tree')
print(mean_absolute_error(val_y, predictions_tree))

print('Mean squared error using Decision Tree')
print(mean_squared_error(val_y, predictions_tree))

print('Median absolute error using Decision Tree')
print(median_absolute_error(val_y, predictions_tree))

Mean error using Decision Tree
0.2546707280922249
Mean squared error using Decision Tree
0.1115613261925844
Median absolute error using Decision Tree
0.1970059880239518


# Try some values to see the output

Average values here to find monthly temperature: https://weatherspark.com/y/42322/Average-Weather-in-Accra-Ghana-Year-Round

In [ ]:
# test with a dummy variable
clo = (random.randint(5,6)/10)
# var = pd.DataFrame({'Outdoor monthly air temperature (C)': [30], 'Relative humidity (%)': [80], 'Air velocity (m/s)': [0.1], 'Clo': [0.7], 'Air temperature (C)' : [25]})
var = pd.DataFrame({'Air temperature (C)': [27], 'Relative humidity (%)': [50], 'Clo': 0.55, 'Outdoor monthly air temperature (C)': [27.5]})
# val = input_transform(var, df_min, df_max)
# print(val)
d = pmv_tree_model_optimized.predict(var)
print(d)


[0.44615385]


# Convert Decision Tree to C

In [ ]:
# export to C DecisionTree, less data 
if __name__ == '__main__':
    c_code = port(pmv_tree_model_optimized)
    
    with open('pmv_tree_model_optimized.h', 'w') as file:
      file.write(c_code)
    files.download('pmv_tree_model_optimized.h')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Export Random Forest to C

In [ ]:
# export to C DecisionTree, less data 
if __name__ == '__main__':
    c_code = port(pmv_forest_model)
    
    with open('pmv_forest_model.h', 'w') as file:
      file.write(c_code)
    files.download('pmv_forest_model.h')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>